# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [64]:
import os
import glob
import psycopg2
import pandas as pd
import datetime
from sql_queries import *

In [65]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [66]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [67]:
song_files = get_files("data/song_data")

In [68]:
len(song_files)

71

In [69]:
song_index = 0

In [70]:
song_files[song_index]

'/home/workspace/data/song_data/A/A/C/TRAACER128F4290F96.json'

In [71]:
filepath = song_files[song_index]

In [72]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [73]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0]
song_data

array(['SOBBUGU12A8C13E95D', 'Setting Fire to Sleeping Giants',
       'ARMAC4T1187FB3FA4C', 2004, 207.77751], dtype=object)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [74]:
song_table_insert

'\nINSERT INTO songs (song_id, title, artist_id, year, duration) VALUES ( %s, %s, %s, %s, %s ) ON CONFLICT (song_id) DO NOTHING;\n'

In [75]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [76]:
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004


In [77]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0]
artist_data

array(['ARMAC4T1187FB3FA4C', 'The Dillinger Escape Plan',
       'Morris Plains, NJ', 40.82624, -74.47995], dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [78]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [79]:
log_files = get_files("data/log_data")

In [80]:
filepath = log_files[0]

In [81]:
df = pd.read_json(filepath, lines=True)
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Celeste,F,0,Williams,NaN,free,"Klamath Falls, OR",GET,Home,1.541078e+12,438,None,200,1541990217796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
1,Pavement,Logged In,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Gary Allan,Logged In,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,None,Logged In,Jacqueline,F,0,Lynch,NaN,paid,"Atlanta-Sandy Springs-Roswell, GA",GET,Home,1.540224e+12,389,None,200,1541990714796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29
5,Charttraxx Karaoke,Logged In,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Fireflies,200,1541990752796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
6,The Libertines,Logged In,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.540224e+12,389,The Good Old Days,200,1541990842796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29
7,Huey Lewis And The News,Logged In,Jacqueline,F,2,Lynch,245.52444,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.540224e+12,389,Hip To Be Square,200,1541991021796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29
8,JOSEF LOCKE & ORCHESTRA,Logged In,Jacqueline,F,3,Lynch,166.26893,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.540224e+12,389,How Can You Buy Killarney (1992 Digital Remaster),200,1541991266796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29
9,Train,Logged In,Jacqueline,F,4,Lynch,216.76363,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.540224e+12,389,Hey_ Soul Sister,200,1541991432796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [93]:
df = df[df['page'] == 'NextSong']

df["timestamp"] = pd.to_datetime(df.ts, unit='ms')
#df["timestamp"] = df.apply(lambda row: datetime.datetime.fromtimestamp(row.ts/1000.0), axis=1)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp
1,Pavement,Logged In,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10,2018-11-12 02:37:38.796
2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53,2018-11-12 02:37:44.796
3,Gary Allan,Logged In,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53,2018-11-12 02:42:21.796
5,Charttraxx Karaoke,Logged In,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Fireflies,200,1541990752796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53,2018-11-12 02:45:52.796
6,The Libertines,Logged In,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.540224e+12,389,The Good Old Days,200,1541990842796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29,2018-11-12 02:47:22.796


Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order
Hint: use pandas' dt attribute to access easily datetimelike properties.

In [102]:
#t = df[["timestamp"]]
t = pd.to_datetime(df.timestamp)


t.head()

1   2018-11-12 02:37:38.796
2   2018-11-12 02:37:44.796
3   2018-11-12 02:42:21.796
5   2018-11-12 02:45:52.796
6   2018-11-12 02:47:22.796
Name: timestamp, dtype: datetime64[ns]

In [103]:
t.head()

1   2018-11-12 02:37:38.796
2   2018-11-12 02:37:44.796
3   2018-11-12 02:42:21.796
5   2018-11-12 02:45:52.796
6   2018-11-12 02:47:22.796
Name: timestamp, dtype: datetime64[ns]

In [104]:
# insert time data records
time_data = (t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday)


In [105]:
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
    
t = pd.to_datetime(df.ts, unit='ms') 

time_data = (t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
    
#t["timestamp"], t["hour"] , t["day"]  = t["timestamp"], t["timestamp"].dt.hour, t["timestamp"].dt.day
#t["week"], t["month"] , t["year"], t["weekday"]  = t["timestamp"].dt.week, t["timestamp"].dt.month, t["timestamp"].dt.year, t["timestamp"].dt.weekday

#time_data = t[["timestamp",'hour', 'day', 'week', 'month','year', 'weekday' ]]
#column_labels = ('timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [101]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.columns = column_labels
time_df.head()

,start_time,hour,day,week,month,year,weekday
1,2018-11-12 02:37:38.796,2,12,46,11,2018,0
2,2018-11-12 02:37:44.796,2,12,46,11,2018,0
3,2018-11-12 02:42:21.796,2,12,46,11,2018,0
5,2018-11-12 02:45:52.796,2,12,46,11,2018,0
6,2018-11-12 02:47:22.796,2,12,46,11,2018,0


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [88]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [89]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [90]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [91]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Pavement,Logged In,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Gary Allan,Logged In,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
5,Charttraxx Karaoke,Logged In,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Fireflies,200,1541990752796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
6,The Libertines,Logged In,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.540224e+12,389,The Good Old Days,200,1541990842796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29


In [92]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    print("%s, %s, %s", row.song, row.artist, row.length)
 
    print(row.song, row.artist, row.length)
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
   
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record only if it's found
    if songid != None and artistid != None:

        cur.execute(songplay_table_insert, (row.timestamp, row.userId, row.level, songid, artistid, row.sessionId,row.location, row.userAgent ))
        conn.commit()

%s, %s, %s Mercy:The Laundromat Pavement 99.16036
Mercy:The Laundromat Pavement 99.16036
%s, %s, %s Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile) Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner 277.15873
Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile) Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner 277.15873
%s, %s, %s Nothing On But The Radio Gary Allan 211.22567
Nothing On But The Radio Gary Allan 211.22567
%s, %s, %s Fireflies Charttraxx Karaoke 225.17506
Fireflies Charttraxx Karaoke 225.17506
%s, %s, %s The Good Old Days The Libertines 179.53914
The Good Old Days The Libertines 179.53914
%s, %s, %s Hip To Be Square Huey Lewis And The News 245.52444
Hip To Be Square Huey Lewis And The News 245.52444
%s, %s, %s How Can You Buy Killarney (1992 Digital Remaster) JOSEF LOCKE & ORCHESTRA 166.26893
How Can You Buy Killarney (1992 Digital Remaster) JOSEF LOCKE & ORCHESTRA 166.26893
%s, %s, %s Hey_ Soul 

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [106]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.